In [1]:
from model import *
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
randomStuff = torch.rand(5, 3, 16, 16)

# conv test

In [3]:
kernel = 2
stride = 3
padding = 1
in_channels = 3
out_channels =48
randomStuff = torch.rand(10, in_channels, 25, 25)
ourConv = Conv2d(in_channels, out_channels, kernel, stride, padding)
torchConv = nn.Conv2d(in_channels, out_channels, kernel, stride, padding)

In [4]:

ourConv.weights = torchConv.weight
ourConv.bias = torchConv.bias

In [5]:
ourOutput = ourConv.forward(randomStuff)
torchOutput = torchConv(randomStuff)
torch.testing.assert_allclose(torchOutput, ourOutput)

AssertionError: Tensor-likes are not close!

Mismatched elements: 38880 / 38880 (100.0%)
Greatest absolute difference: 1.615188181400299 at index (9, 45, 3, 1) (up to 1e-05 allowed)
Greatest relative difference: 21765.155950752393 at index (8, 16, 0, 2) (up to 0.0001 allowed)

In [ ]:
x=Variable(randomStuff,requires_grad=True)
x2=Variable(x*1.0,requires_grad=True)

In [ ]:
x.grad

In [ ]:
o=ourConv.forward(x2)

In [ ]:
p=torchConv(x)

In [ ]:
grdw=torch.rand(p.shape)

In [ ]:


p.backward(grdw)

In [ ]:
a=ourConv.backward(grdw)

In [ ]:
torch.testing.assert_allclose(a, x.grad)

In [ ]:
torch.testing.assert_allclose(torchConv.weight.grad,ourConv.dweights)


In [ ]:
torch.testing.assert_allclose(torchConv.bias.grad,ourConv.dbias)

In [ ]:
torchConv.bias.grad

In [ ]:
ourConv.dbias

# upsample test

In [ ]:
scale_factor = 2
loss=nn.MSELoss()
ourUpsample = NearestUpsampling(scale_factor=scale_factor)
torchUpsample = nn.Upsample(scale_factor = scale_factor)

In [ ]:
ourOutput = ourUpsample.forward(randomStuff)
torchOutput = torchUpsample(randomStuff)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [ ]:
x=Variable(torch.rand(5*3*16).view((5,3,4,4))*1.0,requires_grad=True)

In [ ]:
outp=torch.rand(5*3*8*8).view(5,3,8,8)

In [ ]:
grdout=torch.rand_like(torchUpsample(x))

In [ ]:
from torch.autograd import Variable

In [ ]:
torchUpsample(x).backward(grdout)

In [ ]:
torch.testing.assert_allclose(ourUpsample.backward(grdout),x.grad)

# MSE test

In [ ]:
ourMSE = MSE()
torchMSE = nn.MSELoss()

In [ ]:
x=Variable(randomStuff,requires_grad=True)

In [ ]:
ourOutput = ourMSE.forward(x, torch.zeros_like(x))
torchOutput = torchMSE(x, torch.zeros_like(x))
torch.testing.assert_allclose(torchOutput, ourOutput)

In [ ]:
#test not written correctly
ourBackward = ourMSE.backward()
torchBackward = torchOutput.backward()
torch.testing.assert_allclose(x.grad, ourBackward)

# ReLu test

In [ ]:
ourReLU = ReLU()
torchReLU = nn.ReLU()
x=Variable(randomStuff,requires_grad=True)

In [ ]:
ourOutput = ourReLU.forward(x)
torchOutput = torchReLU(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [ ]:
grdout=torch.rand_like(x)

In [ ]:
grdout.shape

In [ ]:
torchOutput.backward(grdout)

In [ ]:
torch.testing.assert_allclose(ourReLU.backward(grdout),x.grad)

In [ ]:
ourReLU.backward(grdout).shape

# sigmoid test

In [ ]:
ourSigmoid = Sigmoid()
torchSigmoid = nn.Sigmoid()
x=Variable(randomStuff,requires_grad=True)
grdout=torch.rand_like(x)

In [ ]:
ourOutput = ourSigmoid.forward(x)
torchOutput = torchSigmoid(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [ ]:
grdout.shape

In [ ]:
torchOutput.backward(grdout)

In [ ]:
torch.testing.assert_allclose(ourSigmoid.backward(grdout),x.grad)

In [ ]:
ourSigmoid.backward(grdout).shape

# Sequential test

In [ ]:
ourSequential = Sequential(ourConv, ReLU(), Sigmoid())
torchSequential = nn.Sequential( torchConv, nn.ReLU(), nn.Sigmoid())
x=Variable(randomStuff,requires_grad=True)

In [ ]:
ourSequential = Sequential(ourConv)
torchSequential = nn.Sequential( torchConv)
x=Variable(randomStuff,requires_grad=True)

In [ ]:
ourOutput = ourSequential.forward(x)
torchOutput = torchSequential(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [ ]:
grdout=torch.rand_like(ourOutput)

In [ ]:
torchOutput.backward(grdout)

In [ ]:
grdout.shape

In [ ]:
ourSequential.backward(grdout)

In [ ]:
stuff=ourConv.dweights

In [ ]:
torch.testing.assert_allclose(torchConv.weight.grad,stuff)

In [ ]:
torch.testing.assert_allclose(torchConv.bias.grad,ourConv.dbias)

# Full model

In [2]:
torch.set_grad_enabled(False)

In [3]:
noisy_imgs_1, noisy_imgs_2 = torch.load('../data/train_data.pkl') #pairs of images with different noises each time
val_noisy_imgs, clean_imgs = torch.load('../data/val_data.pkl')
val_noisy_imgs = val_noisy_imgs
clean_imgs = clean_imgs
train_input, train_target = ((noisy_imgs_1/255.0)), ((noisy_imgs_2/255.0))


In [4]:
val_imgs_during_training = val_noisy_imgs[800:]
clean_imgs_during_training = clean_imgs[800:]

In [5]:
def psnr ( denoised , ground_truth ) :
    # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
    mse = torch . mean (( denoised - ground_truth ) ** 2)
    return -10 * torch . log10 ( mse + 10** -8)

def validation_check(model):
    with torch.no_grad():
        denoised = ((model(val_imgs_during_training/255)))
        avg_psnr = 0
        for i in range(200) :
             avg_psnr += psnr(denoised[i], clean_imgs_during_training[i]/255)
        return avg_psnr/200

In [6]:
kernel = 3
stride = 2
        
model = Sequential(
    Conv2d(3,48,kernel,stride),
    ReLU(),
    Conv2d(48,48,kernel,stride),
    ReLU(),
    Conv2d(48,48,kernel,stride = 1, padding = 2),
    NNUpsampling(scale_factor = 2),
    Conv2d(48,24,kernel,stride = 1,),
    ReLU(),
    Conv2d(24,24,kernel,stride = 1, padding = 2),
    NNUpsampling(scale_factor = 2),
    Conv2d(24,24,kernel,stride = 1,),
    Conv2d(24,3,kernel,stride = 1,),
    Sigmoid()
)

In [7]:
optimizer = SGD(model.params(), lr = 0.15)

In [8]:
loss = MSE()

In [9]:
import time

In [10]:
epochs = 15
batch_size = 32

for e in range(epochs):
    t = time.time()
    
    for b in range(0, len(train_input), batch_size):
        optimizer.zero_grad()
        data = train_input[b:b+batch_size]
        target = train_target[b:b+batch_size]
        output=loss(model(data),target)
        model.backward(loss.backward())
        optimizer.step()
        
    val = validation_check(model)
    print(e)
    print(val)
    print("epoch time : " + str(time.time() - t))

0
tensor(17.5452)
epoch time : 302.97255182266235
1
tensor(19.2766)
epoch time : 307.9238667488098
2
tensor(19.8543)
epoch time : 310.88910460472107
3
tensor(20.7399)
epoch time : 312.2520868778229
4
tensor(21.1539)
epoch time : 322.1563529968262
5
tensor(21.2533)
epoch time : 322.1631512641907
6
tensor(21.3077)
epoch time : 301.1667289733887
7
tensor(21.4302)
epoch time : 312.9112026691437
8
tensor(21.4753)
epoch time : 354.9637134075165
9
tensor(21.5912)
epoch time : 343.8578779697418
10
tensor(21.7387)
epoch time : 330.95810747146606
11
tensor(21.6489)
epoch time : 333.10935711860657
12
tensor(21.7995)
epoch time : 333.98378109931946
13
tensor(21.8751)
epoch time : 335.01853704452515
14
tensor(21.8754)
epoch time : 332.8360061645508


In [11]:
import pickle
pickle.dump(model,open('bestmodel.pth','wb'))

In [12]:
test=pickle.load(open('bestmodel.pth','rb'))

In [13]:
validation_check(test)

tensor(21.8754)